In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 28 10:01:20 2022

@author: karin
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 27 10:57:03 2022

@author: lvbinghan
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  9 21:43:53 2022

@author: lvbinghan
"""

# importing system libraries

from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest


nltk.download('stopwords')


from nltk.test.classify_fixt import setup_module
setup_module()

import pandas as pd
import numpy as np
import re
import collections

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle












# Read the whole data from the Enron Dataset into a variable allData.

pathwalk = walk(r"/content/gdrive/MyDrive/enron1/")

allHamData, allSpamData = [], []
for root, dr, file in pathwalk:
    if 'ham' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allHamData.append(" ".join(ip.readlines()))
                
    elif 'spam' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allSpamData.append(" ".join(ip.readlines()))
                
                
                

allHamData = list(set(allHamData))
allSpamData = list(set(allSpamData))


hamPlusSpamData = allHamData + allSpamData
labels = ["ham"]*len(allHamData) + ["spam"]*len(allSpamData)

raw_df = pd.DataFrame({"email": hamPlusSpamData, 
                       "label": labels})



raw_df.sample(5)
raw_df['email']

""""""""""""""""""""""""""""""""""""""""""


def preprocess(data):
    # tokenization
    tokens = nltk.word_tokenize(data)
    tokens = [w.lower() for w in tokens if w.isalpha()]

    # finding uncommon words
    cnt = Counter(tokens)
    uncommons = cnt.most_common()[:-int(len(cnt)*0.1):-1]
    
    # listing stopwords from NLTK
    stops = set(nltk.corpus.stopwords.words('english'))

    # removing stop words and uncommon words
    tokens = [w for w in tokens if (w not in stops and w not in uncommons)]

    # lemmatization
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w, pos='a') for w in tokens]

    return tokens

label_encoder = sk.preprocessing.LabelEncoder()
raw_df['label'] = label_encoder.fit_transform(raw_df.label)
import string
from nltk.corpus import stopwords

def process_text(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    return ' '.join([word for word in no_punc.split() if word.lower() not in stopwords.words('english')])

raw_df['email']=raw_df['email'].apply(process_text)
raw_df['email']

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stemming (text):
    return ''.join([stemmer.stem(word) for word in text])
raw_df['email']=raw_df['email'].apply(stemming)
raw_df.head()


from sklearn.feature_extraction.text import CountVectorizer
vectorizer= CountVectorizer()
message_bow = vectorizer.fit_transform(raw_df['email'])

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(message_bow,raw_df['label'],test_size=0.25)

X = message_bow
y = raw_df['label']




# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(message_bow)

# Transform entire BoW into tf-idf corpus
message_bow_tfidf = tfidf_transformer.transform(message_bow)

X_tfidf = message_bow_tfidf
y = raw_df['label']
X_tfidf_train,X_tfidf_test,y_tfidf_train,y_tfidf_test = train_test_split(message_bow_tfidf,raw_df['label'],test_size=0.25)



texts1 = np.asarray(message_bow)
labels1 = np.asarray(raw_df['label'])



import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

import keras
from keras.models import Sequential
from keras.layers import Dense


from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from keras.models import Sequential

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# number of words used as features
max_features = 500000
# cut off the words after seeing 500 words in each document(email)
maxlen = 500


# we will use 80% of data as training, 20% as validation data
training_samples = int(4994 * .8)
validation_samples = int(4994 - training_samples)
# sanity check
print(len(raw_df['email']) == (training_samples + validation_samples))
print("The number of training {0}, validation {1} ".format(training_samples, validation_samples))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_df['email'])
sequences = tokenizer.texts_to_sequences(raw_df['email'])

word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))

data = pad_sequences(sequences, maxlen=maxlen)

print("data shape: ", data.shape)

np.random.seed(42)
# shuffle data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = np.asarray(labels)
labels = y[indices]


texts_train = data[:training_samples]
y_train = labels[:training_samples]
texts_test = data[training_samples:]
y_test = labels[training_samples:]






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True
The number of training 3995, validation 999 
Found 50422 unique words: 
data shape:  (4994, 500)


In [3]:
max_features =500000
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(32, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_ltsm = model.fit(texts_train, y_train, epochs=1, batch_size=10)

pred = (model.predict(texts_test) > 0.5).astype("int32")

acc = model.evaluate(texts_test, y_test)
proba_ltsm = (model.predict(texts_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
print("Test loss is {0:.4f} accuracy is {1:.4f}  ".format(acc[0],acc[1]))

from sklearn.metrics import classification_report
print("LSTM score",classification_report(y_test,pred, digits=4))
print(confusion_matrix(pred, y_test))


32/32 [==============================] - 1s 9ms/step - loss: 0.0607 - acc: 0.9830
Test loss is 0.0607 accuracy is 0.9830  
LSTM score               precision    recall  f1-score   support

           0     0.9916    0.9847    0.9881       718
           1     0.9615    0.9786    0.9700       281

    accuracy                         0.9830       999
   macro avg     0.9766    0.9817    0.9791       999
weighted avg     0.9831    0.9830    0.9830       999

[[707   6]
 [ 11 275]]


512 单层 bach=10 epoch=10 98.0
512 单层 bach=10 epoch=12 96.3
512 单层 bach=10 epoch=10 + dense512  98.2
512 单层 bach=10 epoch=10 + dense256  97.5
512 单层 bach=8 epoch=10 + dense512  98.0
512 单层 bach=12 epoch=10 + dense512  98.6
512 单层 bach=14 epoch=10 + dense512  96.7
512 单层 bach=11 epoch=10 + dense512  97.7
512 单层 bach=12 epoch=20 + dense512  98.1
1024 单层 bach=12 epoch=20 + dense1024  98.2
1024 单层 bach=8 epoch=20 + dense1024  98.4
1024 单层 bach=6 epoch=20 + dense1024  98.1


In [24]:

#98% B32 E20
#98% B64 E20

from keras.layers.core import Flatten



def create_keras_model():

    model2 = Sequential()
    model2.add(Embedding(max_features, 128))
    model2.add(Dropout(0.5))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(Dropout(0.5))
    model2.add(LSTM(128))
    model2.add(Dropout(0.2))
    model2.add(Dense(1, activation='sigmoid'))
    model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])



    return model2


from keras.wrappers.scikit_learn import KerasClassifier

LSTMmodel = KerasClassifier(
        build_fn=create_keras_model, 
        batch_size=64, epochs=20)


LSTMmodel._estimator_type = "classifier"






from sklearn.model_selection import cross_val_score
accuracies_LSTM = cross_val_score(estimator=LSTMmodel, scoring="accuracy", X=data, y=labels, cv=5)

print("LSTM corss validation :  %0.4f accuracy with a standard deviation of %0.4f" % (accuracies_LSTM.mean(), accuracies_LSTM.std()))

Epoch 1/20
32/32 [==============================] - 6s 87ms/step - loss: 0.5434 - acc: 0.7519
Epoch 2/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0985 - acc: 0.9660
Epoch 3/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0324 - acc: 0.9887
Epoch 4/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0787 - acc: 0.9817
Epoch 5/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0154 - acc: 0.9960
Epoch 6/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0084 - acc: 0.9975
Epoch 7/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0050 - acc: 0.9982
Epoch 8/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0035 - acc: 0.9990
Epoch 9/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0026 - acc: 0.9990
Epoch 10/20
32/32 [==============================] - 3s 88ms/step - loss: 0.0016 - acc: 0.9995
Epoch 11/20
32/32 [==============================] - 3s 88m

128单层 b=1 e=20 98.0

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jul 30 16:56:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    33W / 250W |   4667MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------